In [1]:
import pandas as pd
import os
from os.path import join
import psycopg2

from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u

In [20]:
def update_data(conn_str, table, set_columns, set_values, condition=None, condition_values=None):
    """
    Update rows in a PostgreSQL table.

    Parameters:
    - conn_str: PostgreSQL connection string.
    - table: Name of the table you want to update.
    - set_columns: Columns that you want to update.
    - set_values: Values corresponding to the columns you want to update.
    - condition: (Optional) Condition for the rows you want to update.
    - condition_values: (Optional) Values for the conditions.
    
    Returns:
    - Number of updated rows.
    """
    # Form the SQL update statement
    set_clause = ', '.join([f"{col} = %s" for col in set_columns])
    query = f"UPDATE {table} SET {set_clause}"
    if condition:
        query += f" WHERE {condition}"
    
    # Combine the values
    values = set_values
    if condition_values:
        values += condition_values
    
    print(query, values)
    # with psycopg2.connect(conn_str) as conn:
    #     with conn.cursor() as cursor:
    #         cursor.execute(query, values)
    #         conn.commit()
    #         return cursor.rowcount

def delete_data(conn_str, table, condition=None, condition_values=None):
    """
    Delete rows from a PostgreSQL table.

    Parameters:
    - conn_str: PostgreSQL connection string.
    - table: Name of the table from which you want to delete rows.
    - condition: (Optional) Condition to specify which rows to delete.
    - condition_values: (Optional) Values for the condition.
    
    Returns:
    - Number of deleted rows.
    """
    
    # Form the SQL DELETE statement
    query = f"DELETE FROM {table}"
    if condition:
        query += f" WHERE {condition}"
    
    with psycopg2.connect(conn_str) as conn:
        with conn.cursor() as cursor:
            cursor.execute(query, condition_values)
            conn.commit()
            return cursor.rowcount

# Example usage:
conn_str = "dbname='mydatabase' user='username' host='localhost' password='password' port='5432'"

# This will delete rows from the 'users' table where 'id' is 1
deleted_rows = delete_data(conn_str, "users", "id = %s", [1])
print(f"{deleted_rows} row(s) deleted.")

OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "username"
connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "username"


In [2]:
bands = ["R", "G", "I", "Z", "U", "F378", "F395", "F410", "F430", "F515", "F660", "F861"]

In [3]:
def get_filename(field, band):
    return f"{field}_{band}_swp.fits.fz"

In [11]:
def write_log(text):
    open("log.txt", "a").write(text + "\n")

def write_changes(text):
    open("changes.txt", "a").write(text + "\n")

In [5]:
conn = psycopg2.connect(database="mar", user="postgres", password="password", host="127.0.0.1", port="5432")
cursor = conn.cursor()

In [6]:
def querydb(query):
    # Query data
    return pd.read_sql(query, conn)

In [7]:
queryres = querydb("SELECT * from files_individualfile WHERE file_name LIKE '%CN%' ")

/tmp/ipykernel_1644840/2301257939.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


In [8]:
queryres

,id,obsDate,field,file_type,exptime,band,file_name,file_path,ovfile,ovthumb,...,comments,status,isvalid,processed_id,sci_id,superflat_id,raw_sci_id,moonmean,crval1,crval2
0,127221,2019-11-16 05:13:16.914000+00:00,NGC104,SCI,120.0,R,CN2018A76-20191116-021541,20191115/CN2018A76-20191116-021541.fits.fz,None,None,...,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,127212,2019-11-16 04:47:08.612000+00:00,NGC104,SCI,30.0,U,CN2018A76-20191116-014801,20191115/CN2018A76-20191116-014801.fits.fz,None,None,...,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,127219,2019-11-16 05:16:18.822000+00:00,NGC104,SCI,5.0,I,CN2018A76-20191116-021647,20191115/CN2018A76-20191116-021647.fits.fz,None,None,...,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,127211,2019-11-16 04:56:27.792000+00:00,NGC104,SCI,150.0,F395,CN2018A76-20191116-015921,20191115/CN2018A76-20191116-015921.fits.fz,None,None,...,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,127222,2019-11-16 05:09:02.450000+00:00,NGC104,SCI,150.0,G,CN2018A76-20191116-021156,20191115/CN2018A76-20191116-021156.fits.fz,None,None,...,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26467,124564,2019-11-07 06:13:50.712000+00:00,NGC104,SCI,60.0,F660,CN2018A76-20191107-031514,20191106/CN2018A76-20191107-031514.fits.fz,None,None,...,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26468,124582,2019-11-07 06:17:12.134000+00:00,NGC104,SCI,30.0,U,CN2018A76-20191107-031806,20191106/CN2018A76-20191107-031806.fits.fz,None,None,...,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26469,124591,2019-11-07 06:37:08.593000+00:00,NGC104,SCI,30.0,U,CN2018A76-20191107-033802,20191106/CN2018A76-20191107-033802.fits.fz,None,None,...,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26470,127220,2019-11-16 05:12:27.932000+00:00,NGC104,SCI,10.0,R,CN2018A76-20191116-021302,20191115/CN2018A76-20191116-021302.fits.fz,None,None,...,None,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df = pd.read_csv("ref_table_chances.csv")
raw_files_path = "../reductionmedia/TILES/"
raw_files = "../reductionmedia"

coords_table = SkyCoord(ra=df['RA']*u.degree, dec=df['DEC']*u.degree, frame='icrs')

def find_closest_row(ra_input, dec_input):
    input_coord = SkyCoord(ra=ra_input*u.degree, dec=dec_input*u.degree, frame='icrs')
    idx, d2d, d3d = input_coord.match_to_catalog_sky(coords_table)
    if d2d > 700.0 * u.arcsec:
        return None
    return df.iloc[idx]

In [72]:
open("log.txt", "w")

for key, value in df.iterrows():
    field = value["Field"]
    write_log(f"Checking field {field}")
    
    folder_path = join(raw_files_path, value["Field"])
    queryres[(queryres["field"] == field) & (queryres["band"] == "R")]

    for band in bands:
        filedb = queryres[(queryres["field"] == field) & (queryres["band"] == "R")]
        filename = get_filename(field, band)
        filepath = join(folder_path, filename)

        if len(filedb) == 0:
            write_log(f"Not found {filename}")
    
        else:
            write_log(filename + " file_exists:" + str(os.path.exists(filepath)))
            if os.path.exists(filepath):
                header = fits.getheader(filepath, 1)
                print(header)
                ra, dec = float(header['CRVAL1']), float(header['CRVAL2'])
                row = find_closest_row(ra, dec)
                
                print(ra, dec, field)
                print(row)
        
    break

SIMPLE  =                    T / This is a FITS file                            BITPIX  =                  -32                                                  NAXIS   =                    2                                                  NAXIS1  =                11000 / Number of pixels along this axis               NAXIS2  =                11000 / Number of pixels along this axis               EQUINOX =        2000.00000000 / Mean equinox                                   MJD-OBS =       59222.34722222 / Modified Julian date at start                  RADESYS = 'ICRS    '           / Astrometric system                             CTYPE1  = 'RA---TAN'           / WCS projection type for this axis              CUNIT1  = 'deg     '           / Axis unit                                      CRVAL1  =   9.042540856542E+01 / World coordinate on this axis                  CRPIX1  =   5.500500000000E+03 / Reference pixel on this axis                   CD1_1   =  -1.527777777778E-04 / Linear 

In [19]:
queryres.to_csv("CN_db_antes.csv")

In [18]:
open("log.txt", "w")

for key, value in queryres.iterrows():
    field = value["field"]
    
    filepath = join(raw_files, value['file_path'])

    if '.fz' in filepath:
        header = fits.getheader(filepath, 1)
    else:
        header = fits.getheader(filepath, 0)
    
    ra, dec = float(header['CRVAL1']), float(header['CRVAL2'])
    
    contains_value = df['Field'].isin([field]).any()
    if not contains_value:
        continue
    row = find_closest_row(ra, dec)

    #write_log(f"Checking field {field}")
    if row is None or field is None:
        write_log(f"Match error in ref {field}")
        print(row)
        print(field, ra, dec)
        continue
    if field != row['Field']:
        write_log(f"{value['file_name']} {field} -> {row['Field']}")
        write_changes(f"{value['file_name']} {value['id']} -> {row['Field']}")

        print(f"Doing {field}")
        # This will update the 'users' table, setting the 'name' column to 'NewName' where 'id' is 1
        updated_rows = update_data(conn, "files_individualfile", ["field"], [row['Field']], "id = %s", [value['id']])
        print(f"{updated_rows} row(s) updated.")

        updated_rows = update_data(conn, "files_finaltiles", ["status"], [0], "field = %s", [field])
        print(f"{updated_rows} row(s) updated.")

        deleted_rows = delete_data(conn, "files_astrocatalogs", "field = %s", [field])
        print(f"{deleted_rows} row(s) deleted.")

UPDATE files_individualfile SET field = %s WHERE id = %s ['A0085-004', 169074]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A0085-004', 169099]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A0085-004', 169196]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A0085-004', 169113]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A0085-004', 169185]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A0085-004', 169183]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A0085-004', 169122]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A0085-004', 169192]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A0085-004', 169132]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A0085-004', 169165]
None row(s) updated.


UPDATE files_individualfile SET field = %s WHERE id = %s ['A147-007', 232852]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A147-007', 232857]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A147-007', 232870]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A147-007', 232901]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A147-007', 232889]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A147-007', 232838]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A147-007', 232854]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A147-007', 232837]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A147-007', 232843]
None row(s) updated.
UPDATE files_individualfile SET field = %s WHERE id = %s ['A147-007', 232879]
None row(s) updated.
UPDATE fil

In [16]:
# This will update the 'users' table, setting the 'name' column to 'NewName' where 'id' is 1
updated_rows = update_data(conn, "files_individualfile", ["field"], ["TESTE"], "id = %s", [1])

print(f"{updated_rows} row(s) updated.")

UPDATE files_individualfile SET field = %s WHERE id = %s ['TESTE', 1]
None row(s) updated.
